# Connection diagram

This is a poster for the 22nd Workshop on Stochastic Geometry, Stereology and Image Analysis.

It's about the two-point connectivity problem for points on the boundary of a region in the Boolean model.

The picture will be of an ellipse, with two marked points on the boundary, and a path connecting them highlighted (when they're connected by a path).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
plt.rcParams['figure.figsize'] = [10,5]
import networkx
# from PIL import Image, ImageDraw
from matplotlib.patches import Ellipse, Circle
from matplotlib.collections import EllipseCollection
from tqdm import trange
from IPython.display import clear_output
# import colorspace

#### Colours
In this section we choose the diagrams' colours.

In [2]:
GIANT_COLOUR = "#0000ff"      # Blue
SMALL_COLOUR = '#d0d0ff'      # Light blue
X_COLOUR = "#ff5050"          # Red
Y_COLOUR = "#00b300"          # Green
PATH_COLOUR = "#ffd92f"

#### Functions
Here are the functions used in the code

In [11]:
# All the functions live in here.

def sample_in_ellipse(npts, l1=1, l2=1):
    """
    Samples npts iid points uniformly at random in the ellipse with axis lengths 2*l1 and 2*l2.
    Speed's not a big issue, so we just use rejection sampling by taking points in [-l1,l1]x[-l2,l2]
    """
    pts = np.random.uniform(size=(npts,2)) # Uniform in [0,1]^2
    pts *= np.array([2*l1,2*l2])
    pts -= np.array([l1,l2])
    for i, p in enumerate(pts):
        while ( (p[0]/l1)**2 + (p[1]/l2)**2 > 1 ):
            p = np.random.uniform(size=(2)) * np.array([2*l1,2*l2]) - np.array([l1,l2])
        pts[i] = p
    return pts

def get_rgg(points_tree, r):
    """
    Returns the random geometric graph (as a networkx.Graph object)
    made from points with two points connected when their distance is less than 2r
    (i.e. the RGG version of the Boolean model with this point process).
    The vertices are 0, 1, ..., points.shape[0]-1
    """
    # G = networkx.Graph()
    # G.add_nodes_from(range(points_tree.n))
    neighbours = points_tree.query_ball_tree(points_tree,2*r)
    G = networkx.from_dict_of_lists({i:ni for i,ni in enumerate(neighbours)})
    G.add_nodes_from(range(points_tree.n))
    return G

def draw_nopath(points,r,l1,l2,thetax,thetay,rgg):
    fig, ax = plt.subplots()
    region = Ellipse((0,0),2*l1,2*l2,fill=False,linewidth=2)
    ax.add_patch(region)
    ax.set_xlim(-l1-0.5,l1+0.5)
    ax.set_ylim(-l2-0.5,l2+0.5)
    plt.gca().set_aspect(1.0)
    plt.axis('off')
    circlestyle = {'linewidth':None}
    giant_cpt = list(max(networkx.connected_components(rgg),key=len)) # List of balls in the largest component
    xindex = len(points)-2
    yindex = len(points)-1
    if not (xindex in giant_cpt or yindex in giant_cpt):
        # If neither x nor y are part of the giant component
        cptx = list(networkx.node_connected_component(rgg,xindex))
        cpty = list(networkx.node_connected_component(rgg,yindex))
        giant_patch = EllipseCollection(np.full(len(giant_cpt),2*r),np.full(len(giant_cpt),2*r),
                                        np.zeros(len(giant_cpt)),units='x',transOffset=ax.transData,offsets=points[giant_cpt],
                                        facecolors=GIANT_COLOUR)
        ax.add_collection(giant_patch)
        
        cptx_patch = EllipseCollection(np.full(len(cptx),2*r),np.full(len(cptx),2*r),
                                       np.zeros(len(cptx)),units='x',transOffset=ax.transData,offsets=points[cptx],
                                       facecolors=X_COLOUR)
        ax.add_collection(cptx_patch)

        cpty_patch = EllipseCollection(np.full(len(cpty),2*r),np.full(len(cpty),2*r),
                                       np.zeros(len(cpty)),units='x',transOffset=ax.transData,offsets=points[cpty],
                                       facecolors=Y_COLOUR)
        ax.add_collection(cpty_patch)

        small_cpts = [i for i in range(len(points)-2) if not (i in giant_cpt or i in cptx or i in cpty)]
        small_patch = EllipseCollection(np.full(len(small_cpts),2*r),np.full(len(small_cpts),2*r),
                                       np.zeros(len(small_cpts)),units='x',transOffset=ax.transData,offsets=points[small_cpts],
                                       facecolors=SMALL_COLOUR)
        ax.add_collection(small_patch)
        fig.savefig('diagrams/neither_percolates.pdf')
    elif xindex in giant_cpt:
        giant_patch = EllipseCollection(np.full(len(giant_cpt),2*r),np.full(len(giant_cpt),2*r),
                                        np.zeros(len(giant_cpt)),units='x',transOffset=ax.transData,offsets=points[giant_cpt],
                                        facecolors=GIANT_COLOUR)
        ax.add_collection(giant_patch)
        cpty = list(networkx.node_connected_component(rgg,yindex))
        pty_patch = EllipseCollection(np.full(len(cpty),2*r),np.full(len(cpty),2*r),
                                       np.zeros(len(cpty)),units='x',transOffset=ax.transData,offsets=points[cpty],
                                       facecolors=Y_COLOUR)
        ax.add_collection(cpty_patch)
        small_cpts = [i for i in range(len(points)-2) if not (i in giant_cpt or i in cpty)]
        small_patch = EllipseCollection(np.full(len(small_cpts),2*r),np.full(len(small_cpts),2*r),
                                       np.zeros(len(small_cpts)),units='x',transOffset=ax.transData,offsets=points[small_cpts],
                                       facecolors=SMALL_COLOUR)
        ax.add_collection(small_patch)
        fig.savefig('diagrams/x_percolates.pdf')
    elif yindex in giant_cpt:
        giant_patch = EllipseCollection(np.full(len(giant_cpt),2*r),np.full(len(giant_cpt),2*r),
                                        np.zeros(len(giant_cpt)),units='x',transOffset=ax.transData,offsets=points[giant_cpt],
                                        facecolors=GIANT_COLOUR)
        ax.add_collection(giant_patch)
        cptx = list(networkx.node_connected_component(rgg,xindex))
        cptx_patch = EllipseCollection(np.full(len(cptx),2*r),np.full(len(cptx),2*r),
                                       np.zeros(len(cptx)),units='x',transOffset=ax.transData,offsets=points[cptx],
                                       facecolors=X_COLOUR)
        ax.add_collection(cptx_patch)
        small_cpts = [i for i in range(len(points)-2) if not (i in giant_cpt or i in cptx)]
        small_patch = EllipseCollection(np.full(len(small_cpts),2*r),np.full(len(small_cpts),2*r),
                                       np.zeros(len(small_cpts)),units='x',transOffset=ax.transData,offsets=points[small_cpts],
                                       facecolors=SMALL_COLOUR)
        ax.add_collection(small_patch)
        fig.savefig('diagrams/y_percolates.pdf')
    plt.close()
    return

def draw_path(points,r,l1,l2,thetax,thetay,rgg,path):
    fig, ax = plt.subplots()
    region = Ellipse((0,0),2*l1,2*l2,fill=False,linewidth=2)
    ax.add_patch(region)
    ax.set_xlim(-l1-0.5,l1+0.5)
    ax.set_ylim(-l2-0.5,l2+0.5)
    plt.gca().set_aspect(1.0)
    plt.axis('off')
    circlestyle = {'linewidth':None}
    giant_cpt = [i for i in max(networkx.connected_components(rgg),key=len) if not i in path] # List of balls in the largest component
    path = list(path)
    xindex = len(points)-2
    yindex = len(points)-1
    path_patch = EllipseCollection(np.full(len(path),2*r),np.full(len(path),2*r),
                                        np.zeros(len(path)),units='x',transOffset=ax.transData,offsets=points[path],
                                        facecolors=PATH_COLOUR)
    ax.add_collection(path_patch)
    giant_patch = EllipseCollection(np.full(len(giant_cpt),2*r),np.full(len(giant_cpt),2*r),
                                        np.zeros(len(giant_cpt)),units='x',transOffset=ax.transData,offsets=points[giant_cpt],
                                        facecolors=GIANT_COLOUR)
    ax.add_collection(giant_patch)
    small_cpts = [i for i in range(len(points)-2) if not (i in giant_cpt or i in path)]
    small_patch = EllipseCollection(np.full(len(small_cpts),2*r),np.full(len(small_cpts),2*r),
                                       np.zeros(len(small_cpts)),units='x',transOffset=ax.transData,offsets=points[small_cpts],
                                       facecolors=SMALL_COLOUR)
    ax.add_collection(small_patch)
    fig.savefig('diagrams/path.pdf')
    plt.close()
    return

## Finding (and drawing) the shortest path from $x$ to $y$

Now we can create a random geometric graph, we'd like to find the shortest path from $x$ to $y$.

The ellipse has width $2l_1$ and height $2l_2$. Its area is therefore $\pi l_1 l_2$.

In [12]:
# Most of the parameters for the drawing
l1 = 2.5
l2 = 1
thetax = -0.75*np.pi
thetay = 0.25*np.pi
n = 5000
npts = np.random.poisson(lam=n)
print(f'With intensity {n} we have {npts} points.')

points = sample_in_ellipse(npts,l1,l2)
xy = np.array([[l1*np.cos(thetax),l2*np.sin(thetax)], [l1*np.cos(thetay),l2*np.sin(thetay)]])
points = np.append(points,xy,axis=0)
xindex = npts
yindex = npts+1
points_tree = KDTree(points)


intensity = 0.40 # The intensity of the corresponding continuum percolation.
mu = n / (np.pi * l1 * l2) # The intensity of the Poisson process for the Boolean model
r = np.sqrt(intensity / mu)

rgg = get_rgg(points_tree,r)

largest_comp = max(networkx.connected_components(rgg),key=len)
print(f'The largest component has {len(largest_comp)} vertices, {100*len(largest_comp)/npts:.1f} percent of the total vertices.')
try:
    path = networkx.shortest_path(rgg,xindex,yindex)
    print(f'There is a path of length {len(path)}')
    is_path = True
except Exception as e:
    print(f'No path from x to y at radius {r:.4f}')
    is_path = False
    if xindex in largest_comp:
        print("x is in the giant component")
    elif yindex in largest_comp:
        print("y is in the giant component")
    else:
        print("Neither x nor y is in the giant component")

With intensity 5000 we have 4897 points.
The largest component has 4116 vertices, 84.1 percent of the total vertices.
No path from x to y at radius 0.0251
y is in the giant component


In [13]:
if is_path:
    draw_path(points,r,l1,l2,thetax,thetay,rgg,path)
else:
    draw_nopath(points,r,l1,l2,thetax,thetay,rgg)

In [14]:
# Try to get a path
count = 0
while True:
    l1 = 2.5
    l2 = 1
    thetax = -0.75*np.pi
    thetay =  0.25*np.pi
    n = 5000 # n = mu * area(ellipse)
    npts = np.random.poisson(lam=n)
    
    points = sample_in_ellipse(npts,l1,l2)
    xy = np.array([[l1*np.cos(thetax),l2*np.sin(thetax)], [l1*np.cos(thetay),l2*np.sin(thetay)]])
    points = np.append(points,xy,axis=0)
    xindex = npts
    yindex = npts+1
    points_tree = KDTree(points)

    mu = n / (np.pi*l1*l2) # The intensity of the arrivals process.
    intensity = 0.40 # The intensity of the corresponding continuum percolation. The critical point is around 0.36.
    r = np.sqrt(intensity / mu)
    
    rgg = get_rgg(points_tree,r)
    
    largest_comp = max(networkx.connected_components(rgg),key=len)
    print(f'The largest component has {str(len(largest_comp)).zfill(4)} vertices, {100*len(largest_comp)/npts:.1f} percent of the total vertices.')
    try:
        path = networkx.shortest_path(rgg,xindex,yindex)
        print(f'There is a path of length {len(path)}, after {count+1} attempts')
        break
    except Exception as e:
        count += 1
        print(f'No path on attempt {count}')
        clear_output(wait=True)
        continue
draw_path(points,r,l1,l2,thetax,thetay,rgg,path)

The largest component has 4362 vertices, 86.8 percent of the total vertices.
There is a path of length 133, after 2 attempts


## Specific diagrams

I'd like to generate big batches of diagrams with given parameters so I can pick the best one.

In [17]:
import os

l1 = 2.5
l2 = 1
thetax = -0.75*np.pi
thetay = 0.25*np.pi
xy = np.array([[l1*np.cos(thetax),l2*np.sin(thetax)], [l1*np.cos(thetay),l2*np.sin(thetay)]])
n = 5000
folder = 'n5k'
intensity = 0.40 # The intensity of the corresponding continuum percolation.
mu = n / (np.pi * l1 * l2) # The intensity of the Poisson process for the Boolean model

In [18]:
def x_percolates_diagram(filename):
    while True:
        npts = np.random.poisson(lam=n)
        # print(f'With intensity {n} we have {npts} points.')
        points = sample_in_ellipse(npts,l1,l2)
        points = np.append(points,xy,axis=0)
        xindex = npts
        yindex = npts+1
        points_tree = KDTree(points)
        r = np.sqrt(intensity / mu)
        rgg = get_rgg(points_tree,r)
        largest_comp = max(networkx.connected_components(rgg),key=len)
        if not xindex in largest_comp:
            continue
        # print(f'The largest component has {len(largest_comp)} vertices, {100*len(largest_comp)/npts:.1f} percent of the total vertices.')
        try:
            path = networkx.shortest_path(rgg,xindex,yindex)
        except Exception as e:
            draw_nopath(points,r,l1,l2,thetax,thetay,rgg)
            os.rename('diagrams/x_percolates.pdf', f'diagrams/{filename}.pdf')
            break

def neither_percolates_diagram(filename):
    while True:
        npts = np.random.poisson(lam=n)
        # print(f'With intensity {n} we have {npts} points.')
        points = sample_in_ellipse(npts,l1,l2)
        points = np.append(points,xy,axis=0)
        xindex = npts
        yindex = npts+1
        points_tree = KDTree(points)
        r = np.sqrt(intensity / mu)
        rgg = get_rgg(points_tree,r)
        largest_comp = max(networkx.connected_components(rgg),key=len)
        if xindex in largest_comp or yindex in largest_comp:
            continue
        # print(f'The largest component has {len(largest_comp)} vertices, {100*len(largest_comp)/npts:.1f} percent of the total vertices.')
        try:
            path = networkx.shortest_path(rgg,xindex,yindex)
        except Exception as e:
            draw_nopath(points,r,l1,l2,thetax,thetay,rgg)
            os.rename('diagrams/neither_percolates.pdf', f'diagrams/{filename}.pdf')
            break

def path_diagram(filename):
    while True:
        npts = np.random.poisson(lam=n)
        # print(f'With intensity {n} we have {npts} points.')
        points = sample_in_ellipse(npts,l1,l2)
        points = np.append(points,xy,axis=0)
        xindex = npts
        yindex = npts+1
        points_tree = KDTree(points)
        r = np.sqrt(intensity / mu)
        rgg = get_rgg(points_tree,r)
        largest_comp = max(networkx.connected_components(rgg),key=len)
        # print(f'The largest component has {len(largest_comp)} vertices, {100*len(largest_comp)/npts:.1f} percent of the total vertices.')
        try:
            path = networkx.shortest_path(rgg,xindex,yindex)
            draw_path(points,r,l1,l2,thetax,thetay,rgg,path)
            os.rename('diagrams/path.pdf', f'diagrams/{filename}.pdf')
            break
        except Exception as e:
            continue

In [19]:
max_diagrams = 20
progress = trange(max_diagrams,leave=False)
for i in progress:
    # progress.set_description('First draw the "x percolates" diagram')
    # x_percolates_diagram(f'{folder}/xperc{str(i).zfill(3)}')
    progress.set_description('Now the "neither percolates" diagram ')
    neither_percolates_diagram(f'{folder}/noperc{str(i).zfill(3)}')
    # progress.set_description('Finally, the "path x <-> y" diagram  ')
    # path_diagram(f'{folder}/path{str(i).zfill(3)}')
print("Done!")

Done!
